# webscrape debug

In [1]:
try:
    import win32com.client
except:
    print("not on windows os: win32com.client not imported.")
    
import pandas as pd
import numpy as np
import datetime as dt
import pprint
import json
import time
import math
import sys
import re
import os

base = os.getcwd()
pp = pprint.PrettyPrinter(indent = 1)
print("imported modules successfully.")

not on windows os: win32com.client not imported.
imported modules successfully.


In [2]:
#os.chdir(os.path.join(basepath, 'Logins'))
try:
    global inbox
    
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.Folders('ENGIENA-GSERNASERVICES (ENGIE North America)')
    folder = inbox.Folders("Inbox")
    messages = folder.Items

    aaron_filter = "[SenderEmailAddress] = 'aaron.downing@eversource.com'"
    admin_filter = "[SenderEmailAddress] = 'epoadmin@eversource.com'"
    ngrid_filter = "[SenderEmailAddress] = 'michael.stanton@nationalgrid.com'"

    today = dt.datetime.now()

    aaron = messages.Restrict(aaron_filter)
    admin = messages.Restrict(admin_filter)
    ngrid = messages.Restrict(ngrid_filter)
    
    print(len(aaron), 'aaron emails,', len(admin), 'epo emails', len(ngrid), 'ngrid emails')
    
    j = 0
    error = 0
    scrape = {}
    

    #scrape, j2 = iter_mail(aaron_to_dict, aaron, j)
        
    print('scraping emails...')
    #start iterating through emails
    mail = aaron.GetLast()
    i = j
    try:
        print('init email')
        msg_row = admin_to_dict(mail.Body)
        msg_row.append(('date', str_to_date(mail.ReceivedTime)))

        master = [(i, dict(msg_row))]
    except:
        master = []

    i += 1
   
    while mail:

        mail = aaron.GetPrevious()

        try:
            print('next email')
            msg_row = admin_to_dict(mail.Body)
            msg_row.append(('date', str_to_date(mail.ReceivedTime)))

            msg_row = dict(msg_row)

            new = (i, msg_row)

            master.append(new)

            i += 1


        except:
            pass

    master = dict(master)

except:
    print("job failed.")

job failed.


In [3]:
master

NameError: name 'master' is not defined

In [3]:
basepath = os.getcwd()

def flatten(l):
    out = []
    for item in l:
        if isinstance(item, (list, tuple)):
            out.extend(flatten(item))
        else:
            if item != '' and item != ' ':
                out.append(item)
    return out

def clean_acct(acct):
    return ''.join(acct.split(' - '))

def str_to_date(datestring):
    return (dt.datetime.strptime(str(datestring).split('+')[0],"%Y-%m-%d %H:%M:%S"))

def date_to_str(datetime_obj):
    return (dt.datetime.strftime(datetime_obj, format = '%m/%d/%Y %H:%M:%S'))

def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res 

def ba_sa_merge(a, b): 
    merged = []
    
    for k in range(0, len(a)):
        m_acct = a[k] + '_' + b[k]
        merged.append(m_acct)
    return merged


def aaron_to_dict(body):
    
    parse = body.split('\n')
    parse = [thing.splitlines() for thing in parse]

    parse2 = flatten(parse)
    strp_len = [len(item.strip(' ')) for item in parse2]

    acct_start = parse2.index('Account Number(s): ') + 1
    acct_end = strp_len.index(0)

    for item in parse2:

        split_item = item.split(': ')
        if split_item[0]  == 'User Id':
            user = ('user', split_item[1])
        
        elif split_item[0] == 'Password':
            pw = ('pw', split_item[1])
            
        elif split_item[0] == 'Customer':
            name = ('name', split_item[1])
        
        
    accts = parse2[acct_start:acct_end]
    clean_accts = [clean_acct(acct) for acct in accts]
    accts_tup = ('accts', clean_accts)
    row = [user, pw, name, accts_tup]
    return row


def admin_to_dict(body):
    
    parse = body.split('\n')
    parse = [thing.splitlines() for thing in parse]

    parse2 = flatten(parse)
    
    if (len(body) > 2500) or ("SA Number" in body):

        try:
            ba_list = [''.join(elem.split('-')) for elem in parse2[9].split(' ')]

            sa_list = [elem for elem in parse2[10].split(' ')]
            
            accts = ('accts', ba_sa_merge(ba_list, sa_list))

            for item in parse2:

                split_item = item.split(': ')
                if split_item[0]  == 'User Id':
                    user = ('user', split_item[1])
        
                elif split_item[0] == 'Password':
                    pw = ('pw', split_item[1])
            
                elif split_item[0] == 'Customer':
                    name = ('name', split_item[1])
                    
            row = [user, pw, name, accts]
                
        except:
            
            for index, item in enumerate(parse2):
                if item.lower() == 'user id':
                    special_index = index

            name = ('name', parse2[special_index + 1])
            ba_list = parse2[special_index + 2].split(' ')
            sa_list = parse2[special_index + 3].split(' ')
            user = ('user', parse2[special_index + 4])
            pw = ('pw', parse2[special_index + 6])

            ba_list = [ba.replace('-', '') for ba in ba_list]
                    
            accts = ('accts', ba_sa_merge(ba_list, sa_list))
                    
            row = [user, pw, name, accts]        
                    
                    
                    

    else:
        for item in parse2:

            split_item = item.split(': ')
        
            if split_item[0]  == 'User ID':
                user = ('user', split_item[1])
        
            elif split_item[0] == 'Password':
                pw = ('pw', split_item[1])
            
            elif split_item[0] == 'Customer':
                name = ('name', split_item[1])

            elif split_item[0] == 'Account Number':
                acct_list = [str(acct) for acct in split_item[1].split(' ')]
                accts = ('accts', acct_list)
        
        row = [user, pw, name, accts]
    
    return(row)

def ngrid_to_dict(body):
    
    parse = body.split('\n')
    parse = [thing.splitlines() for thing in parse]

    parse2 = flatten(parse)
    accts_list = []
    names_list = []
        
    for item in parse2:
        split_item = item.split(': ')
        if split_item[0] == 'Userid':
            username = split_item[1].replace(' ', '')
            user = ('user', username)
        
        if split_item[0] == 'Password':
            password = split_item[1].replace(' ', '')
            pw = ('pw', password)
        
        try:
            split_item2 = [thing.split(' ') for thing in split_item][0]
            split_item3 = list(filter(None, split_item2))
            int(split_item3[0])
            accts_list.append(split_item3[0])
            full_name = ' '.join(split_item3[1:])
            names_list.append(full_name)
        
        except:
            pass
        
    accts = ('accts', accts_list)
    name = ('name', names_list)
        
    row = [user, pw, name, accts]
    return(row)


def iter_mail(sender_func, mailbox, index):

    print('scraping emails...')
    #start iterating through emails
    mail = mailbox.GetLast()
    i = index
    try:
        msg_row = sender_func(mail.Body)
        msg_row.append(('date', str_to_date(mail.ReceivedTime)))

        master = [(i, dict(msg_row))]
    except:
        master = []
    
    i += 1
   
    while mail:
        
        mail = mailbox.GetPrevious()
        
        try:
            msg_row = sender_func(mail.Body)
            msg_row.append(('date', str_to_date(mail.ReceivedTime)))
            
            msg_row = dict(msg_row)

            new = (i, msg_row)
        
            master.append(new)
            
            i += 1
            
        
        except:
            pass
            
    master = dict(master)
            
    return(master, i)

    


# ### Get Filtered Messages (Sender, Date, Body)

# aaron_to_dict will gives a dictionary for each account of the structure  
#   
#     {'user': 'nhengi-ston3n',
#          'pw': 'fm54f7',
#          'name': 'STONEWALL KITCHEN LLC',
#          'accts': ['800531501']}  
#  
#  The following code will add the date email is received (if within last week) w/ structure: 
#    
#      {'user': 'nhengi-ston3n',
#          'pw': 'fm54f7',
#          'name': 'STONEWALL KITCHEN LLC',
#          'accts': ['800531501'],
#          'date': datetime.datetime(2019, 5, 16, 10, 17, 21)}
#  
#  nest these dictionaries into a scary looking dictionary:
#  
#      {0: {'user': 'nhengi-ston3n', 
#             'pw': 'fm54f7', 
#             'name': 'STONEWALL KITCHEN LLC', 
#             'accts': ['800531501'], 
#             'date': datetime.datetime(2019, 5, 16, 10, 17, 21)}, 
#       1: {'user': 'nhengi-morg1n', 
#             'pw': '6z2s5e', 
#             'name': 'MORGAN ADVANCED CERAMICS', \
#             'accts': ['800514701'], 
#             'date': datetime.datetime(2019, 4, 17, 12, 34, 30)}, 
#       2: {'user': 'nhengi-ferr1n', 
#             'pw': 'xx27b4', 
#             'name': 'FERROTEC AMERICA CORP', 
#             'accts': ['800511301'], 
#             'date': datetime.datetime(2019, 4, 17, 12, 7, 55)},
#         
#             #.....

#https://docs.microsoft.com/en-us/office/vba/api/outlook.mailitem

def get_emails():

    global inbox
    
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.Folders('ENGIENA-GSERNASERVICES (ENGIE North America)')
    folder = inbox.Folders("Inbox")
    messages = folder.Items

    aaron_filter = "[SenderEmailAddress] = 'aaron.downing@eversource.com'"
    admin_filter = "[SenderEmailAddress] = 'epoadmin@eversource.com'"
    ngrid_filter = "[SenderEmailAddress] = 'michael.stanton@nationalgrid.com'"

    today = dt.datetime.now()

    aaron = messages.Restrict(aaron_filter)
    admin = messages.Restrict(admin_filter)
    ngrid = messages.Restrict(ngrid_filter)
    
    print(len(aaron), 'aaron emails,', len(admin), 'epo emails', len(ngrid), 'ngrid emails')
    
    j = 0
    error = 0
    
    try:
        print('parsing aaron.downing@eversource.com inbox')
        print('')
        scrape, j2 = iter_mail(aaron_to_dict, aaron, j)
        
    except:
        print('error parsing aaron')
        error += 1
        
    
    try:
        print('parsing epoadmin@eversource.com')
        print('')
        admin_scrape, j3 = iter_mail(admin_to_dict, admin, j2)
        master = Merge(scrape, admin_scrape)
        
    except:
        print('error parsing EPO admin')
        error += 1
        master = {}
    
    try:
        print('parsing michael.stanton@nationalgrid.com')
        print('')
        ngrid_scrape, j4 = iter_mail(ngrid_to_dict, ngrid, j3)
        master = Merge(master, ngrid_scrape)
        
    except:
        print('')
        print('error parsing ngrid')
        error += 1
        if error == 3:
            pass
    

    print('error with', error, 'of 3 inboxes')
    print('found ', len(master.keys()), ' new emails: ')
    
    pretty_json = json.dumps(master, default = lambda date: date_to_str(date), sort_keys = True, indent = 4)
    lame_json = json.dumps(master, default = lambda date: date_to_str(date), sort_keys = True)

    json_name = 'email_bodies_' + date_to_str(today).split(' ')[0].replace('/', '_') + '.json'

    print('writing .json object')
    
    os.chdir(os.path.join(basepath, 'Logins'))
    
    with open(json_name, 'w') as f:
        json.dump(lame_json, f)
    
    #print(pretty_json)
    return master, json_name
 
    
def bodies_json(bodies):

    test = pd.DataFrame.from_dict(bodies, orient = 'index')

    if type(test.date[0]) == str:
            test.date = pd.to_datetime(test.date)

    accts_success = [len(accts) > 0 for accts in test.accts]
    accts_fail = [not val for val in accts_success]
        
    good = test[accts_success].reset_index(drop = True)
    util = []
    
    for i, a in enumerate(good.accts):
        first_acct = a[0]
        leading = a[0][:2]
        
        if leading == '80':
            util.append('PSNH')
            
        elif leading == '51' and (len(first_acct.split('_')) > 1):
            util.append('CLP')
            
        elif leading == '54' and (len(first_acct.split('_')) > 1):
            util.append('WMECO')
                
        else:
            if 'SUEZ' in good.user[i]:
                util.append('NGRID')
                
            else:
                util.append('NSTAR')
            
    good['util'] = util
        

    email_error = []

    if len(accts_fail) > 0:
        bad = test[accts_fail].reset_index()
        mail_error = 'EMAIL_SCRAPE_ERROR.csv'

        bad.to_csv(mail_error, header = True, index = False)

    return(good, bodies)


print("defined functions.")

defined functions.


# debug

In [4]:
os.getcwd()

'/Users/stevenhurwitt/Documents/dealersocket-speedtest/IDR_Drop'

In [6]:
! ls

Downloads                          Raw_IDR
EPOwebscrape.py                    __init__.py
Eversource_nonEDI_IDR_requests.pdf __init____20200120035935.py
IDR                                __pycache__
IDR.zip                            chromedriver
IDR_Drop.ipynb                     chromedriver.exe
IDR_Drop_Portal.ipynb              csv_to_lse
IDR_drop_gui.py                    db_connect.py
IDRdrop.py                         emailscrape.py
IDRdrop__20200120035935.py         init.git
Logins                             init__20200120035935.git
PSNH_split.ipynb                   webscrape_debug.ipynb


In [14]:
# ! touch __init__.py

In [19]:
! pip install cx_Oracle

     |████████████████████████████████| 359 kB 4.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cx-Oracle: filename=cx_Oracle-8.2.1-cp38-cp38-macosx_11_0_arm64.whl size=180855 sha256=01522ba54e934a02e63c829c4312122d4723de66dc87eacdc3bd63b799933e96
  Stored in directory: /Users/stevenhurwitt/Library/Caches/pip/wheels/4a/24/ca/0ce823a8a0ee3e9adc1368bb8a5109ffec269728dcee2f60a0
Successfully built cx-Oracle


In [5]:
try:
    import db_connect as db
except:
    print("failed to import db_connect.")
try:
    import emailscrape
except:
    print("failed to import emailscrape.")
try:
    import IDRDrop
except:
    print("failed to import IDRDrop.")

print("imported custom modules successfully.")

failed to import db_connect.
not on windows os: failed to import win32com.client.
failed.
failed to import emailscrape.
failed to import IDRDrop.
imported custom modules successfully.


In [8]:
os.getcwd()

'/Users/stevenhurwitt/Documents/dealersocket-speedtest/IDR_Drop'

In [6]:
base = os.getcwd()
print(base)

base_split = base.split("/")
print(base_split)

new_base = base_split[:5]
print(new_base)
new_base2 = "/".join(new_base)
print(new_base2)
try:
    os.chdir(new_base2)
except:
    print("changed to base: {}.".format(new_base2))

try:
    os.chdir('/Users/stevenhurwitt/Documents/dealersocker-speedtest/')
except FileNotFoundError as e:
    print(e)

/Users/stevenhurwitt/Documents/dealersocket-speedtest/IDR_Drop
['', 'Users', 'stevenhurwitt', 'Documents', 'dealersocket-speedtest', 'IDR_Drop']
['', 'Users', 'stevenhurwitt', 'Documents', 'dealersocket-speedtest']
/Users/stevenhurwitt/Documents/dealersocket-speedtest
[Errno 2] No such file or directory: '/Users/stevenhurwitt/Documents/dealersocker-speedtest/'


In [7]:
! ls

IDR_Drop         README.md        requirements.txt
LICENSE          dealersocket


## IDR_Drop

In [11]:
import IDR_Drop

In [12]:
dir(IDR_Drop)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

## dealersocket

In [13]:
import dealersocket

In [14]:
dir(dealersocket)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

## pyspark

In [8]:
! conda env list

# conda environments:
#
                         /Users/stevenhurwitt/mambaforge
                         /Users/stevenhurwitt/mambaforge/envs/miniforge
                         /Users/stevenhurwitt/mambaforge/envs/reddit
base                     /Users/stevenhurwitt/miniforge3
dbt                      /Users/stevenhurwitt/miniforge3/envs/dbt
dbx-p40                  /Users/stevenhurwitt/miniforge3/envs/dbx-p40
dbx-p58                  /Users/stevenhurwitt/miniforge3/envs/dbx-p58
dealersocket          *  /Users/stevenhurwitt/miniforge3/envs/dealersocket
flask                    /Users/stevenhurwitt/miniforge3/envs/flask
kafka                    /Users/stevenhurwitt/miniforge3/envs/kafka
reddit                   /Users/stevenhurwitt/miniforge3/envs/reddit
speedtest                /Users/stevenhurwitt/miniforge3/envs/speedtest
                         /opt/homebrew/Caskroom/miniconda/base
                         /opt/homebrew/anaconda3



In [10]:
# ! conda activate dbx-p40

In [11]:
# ! python3.7 -m ipykernel install --user --name=dbx-p40

In [12]:
# ! python3.7 -m ipykernel install --user --name=dbx-p58

In [13]:
# ! python3.7 -m ipykernel install --user --name=dealersocket

In [14]:
# ! python3.7 -m ipykernel install --user --name=reddit

In [15]:
# ! python3.7 -m ipykernel install --user --name=kafka

In [16]:
# ! python3.7 -m ipykernel install --user --name=flask

In [17]:
# ! python3.7 -m ipykernel install --user --name=speedtest

In [18]:
# ! python3.7 -m ipykernel install --user --name=dbt

In [20]:
! source ~/.zshrc

In [22]:
! ls

IDR_Drop         README.md        requirements.txt
LICENSE          dealersocket


In [23]:
print(base)

/Users/stevenhurwitt/Documents/dealersocket-speedtest/IDR_Drop


## ch dir

In [24]:
os.chdir("/Users/stevenhurwitt/Documents")

In [25]:
! ls

$RECYCLE.BIN                        dealersocket-speedtest
KafkaRedditStream                   desktop.ini
KafkaRedditStream-og                la-tools-test
SQL Server Management Studio        python-sample-vscode-flask-tutorial
Visual Studio 2017                  speedtest
austin2017.jpeg                     steven-headshot.jpeg
bpDataAnalyticsToolkit-DBX


## KafkaRedditStream

In [26]:
cd KafkaRedditStream

/Users/stevenhurwitt/Documents/KafkaRedditStream


In [27]:
! ls

Dockerfile              dbx-workspaces.json     reddit
Dockerfile.old          dbx_tax_ala             reddit-env
Dockerfile.superold     dist                    reddit.egg-info
README.md               docker-compose.yml      reddit.ipynb
README2.md              docker.txt              setup.py
README_DOCKER.md        docs                    sql-server.txt
README_WHEEL.md         env                     src
__pycache__             failed-test-results.txt startup.sh
assets                  hi.txt                  test-results.txt
build                   jar-dir.json            test.json
config.yaml             kafka                   test_xdb
dbt.yml                 license.txt             tests
dbt_project.yml         macros
dbx-configure.txt       profiles.yml


## reinstall wheel

In [30]:
! pip install ./dist/reddit-0.0.1-py2.py3-none-any.whl --force-reinstall

Processing ./dist/reddit-0.0.1-py2.py3-none-any.whl


## pytest

In [31]:
! pytest -v ./src/test/python/test_suite.py

============================= test session starts ==============================
platform darwin -- Python 3.8.11, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /Users/stevenhurwitt/miniforge3/envs/dealersocket/bin/python
cachedir: .pytest_cache
rootdir: /Users/stevenhurwitt/Documents/KafkaRedditStream
collected 20 items                                                             

src/test/python/test_suite.py::test_imports_ok FAILED                    [  5%]
src/test/python/test_suite.py::test_token_headers_has_keys FAILED        [ 10%]
src/test/python/test_suite.py::test_token_body_has_bearer FAILED         [ 15%]
src/test/python/test_suite.py::test_request_gets_200 FAILED              [ 20%]
src/test/python/test_suite.py::test_request_data_length FAILED           [ 25%]
src/test/python/test_suite.py::test_pandas_df FAILED                     [ 30%]
src/test/python/test_suite.py::test_numpy_stats PASSED                   [ 35%]
src/test/python/test_suite.py::test_os_base PASSED         

In [21]:
! databricks-connect test

zsh:1: command not found: databricks-connect


In [145]:
! python3.7 -m pip install findspark

In [146]:
! python3.7 -m pip install "databricks-connect==7.3.*"

In [149]:
# import pyspark

In [143]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

## pandas

In [49]:
import pandas as pd

## numpy

In [50]:
import numpy as np

## selenium

In [51]:
import selenium

## beautifulsoup4

In [53]:
import bs4 as beautifulsoup4

## chromedriver

In [54]:
! which chromedriver

/opt/homebrew/bin/chromedriver


## docker

In [55]:
! which docker

/usr/local/bin/docker


In [56]:
! docker login

Authenticating with existing credentials...
Login Succeeded


## sql server edge

### az-cli

In [57]:
! which az

/opt/homebrew/bin/az


In [59]:
# ! az login

# The default web browser has been opened at https://login.microsoftonline.com/common/oauth2/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
# You have logged in. Now let us find all the subscriptions to which you have access...
# [
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "f55d4ef9-4d7f-4763-8661-9b82de6c08c9",
#     "isDefault": true,
#     "managedByTenants": [
#       {
#         "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
#       }
#     ],
#     "name": "zne-udl1-p-ust0-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "3dc3f829-f5db-4014-b75f-1718ffb337b2",
#     "isDefault": false,
#     "managedByTenants": [
#       {
#         "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
#       }
#     ],
#     "name": "zne-usw0-n-idm-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "ae3f2be6-92ad-4a3a-9f90-baf1e9b3562c",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zsc-evcs-p-mswvd-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "6539e0ec-09d4-4b35-9bd0-d35dba747572",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zsc-envd-p-ras-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "3a7b4fea-df52-4b37-92c1-3ed9aafac9fb",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zne-envd-p-ras-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "b4c9bbf7-fef0-49e3-b326-1b80149ebe0c",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zsa-edwp-p-apvcs-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "2b310ddf-3296-456b-92f8-5cfff2f8e8c2",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zne-evcs-p-dwp-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "eb118bf0-7368-4d05-ba02-9cf31cb431f7",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zne-ewpe-p-wndgms-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "1acbaf85-58aa-4f0a-a93b-9176a0f08bd3",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zwe-ess1-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "7d5f9f0d-5f8a-4159-9fd5-42435ad8b764",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zne-eds1-n-test-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "5c42403b-0ffd-4d5c-a477-b74227af91ac",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "znc-ess1-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "4fd3bb7a-ecde-46f0-89d8-124cd5a7b075",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zne-ess1-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "719f5bc7-faa1-4724-95ac-d0153e84c8f5",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zsc-ess1-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "26518870-f42b-4800-8f93-ce687f3bd9a4",
#     "isDefault": false,
#     "managedByTenants": [
#       {
#         "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
#       }
#     ],
#     "name": "zsc-udl0-n-ust0-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "0af2cfef-5d80-4171-ac37-2c2bded97551",
#     "isDefault": false,
#     "managedByTenants": [
#       {
#         "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
#       }
#     ],
#     "name": "zne-udl0-n-ust0-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   },
#   {
#     "cloudName": "AzureCloud",
#     "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "id": "19a7a8f0-6b4f-460e-9df1-11784b01b6e8",
#     "isDefault": false,
#     "managedByTenants": [],
#     "name": "zsa-ess1-p-sbc",
#     "state": "Enabled",
#     "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
#     "user": {
#       "name": "Steven.Hurwitt@bp.com",
#       "type": "user"
#     }
#   }
# ]

### docker container

In [60]:
! docker pull mcr.microsoft.com/azure-sql-edge

Using default tag: latest
latest: Pulling from azure-sql-edge
Digest: sha256:209a502ac7d35a8e9d5ae777bca874930950a6bf0ec4915acf23390321c576e9
Status: Image is up to date for mcr.microsoft.com/azure-sql-edge:latest
mcr.microsoft.com/azure-sql-edge:latest


### run docker

In [63]:
! docker run mcr.microsoft.com/azure-sql-edge:latest --accept-eula -y

/opt/mssql/bin/permissions_check.sh: line 68: exec: --: invalid option
exec: usage: exec [-cl] [-a name] [command [arguments ...]] [redirection ...]
Azure SQL Edge will run as non-root by default.
This container is running as user mssql.
To learn more visit https://go.microsoft.com/fwlink/?linkid=2140520.


## python version

In [70]:
! python --version
! pip --version

Python 3.8.11
pip 21.2.2 from /Users/stevenhurwitt/miniforge3/envs/dealersocket/lib/python3.8/site-packages/pip (python 3.8)


## alias

In [76]:
! alias python=python3.7
! alias pip="python3.7 -m pip"
print("set aliases.")

set aliases.


## dependencies

#### python3.7

In [78]:
! python3.7 --version

Python 3.7.10 (77787b8f4c49, May 19 2021, 05:28:43)
[PyPy 7.3.5 with GCC Apple LLVM 12.0.5 (clang-1205.0.22.9)]


#### pip3.7

In [79]:
! python3.7 -m pip --version

pip 21.2.4 from /Users/stevenhurwitt/.pyenv/versions/pypy3.7-7.3.5/site-packages/pip (python 3.7)


#### java1.8

In [81]:
! java -version

openjdk version "1.8.0_302"
OpenJDK Runtime Environment (Zulu 8.56.0.23-CA-macos-aarch64) (build 1.8.0_302-b08)
OpenJDK 64-Bit Server VM (Zulu 8.56.0.23-CA-macos-aarch64) (build 25.302-b08, mixed mode)


#### scala2.13

In [82]:
! scala -version

Scala code runner version 2.13.6 -- Copyright 2002-2021, LAMP/EPFL and Lightbend, Inc.


## conda

In [80]:
! conda env list

# conda environments:
#
                         /Users/stevenhurwitt/mambaforge
                         /Users/stevenhurwitt/mambaforge/envs/miniforge
                         /Users/stevenhurwitt/mambaforge/envs/reddit
base                     /Users/stevenhurwitt/miniforge3
dbt                      /Users/stevenhurwitt/miniforge3/envs/dbt
dbx-p40                  /Users/stevenhurwitt/miniforge3/envs/dbx-p40
dbx-p58                  /Users/stevenhurwitt/miniforge3/envs/dbx-p58
dealersocket          *  /Users/stevenhurwitt/miniforge3/envs/dealersocket
flask                    /Users/stevenhurwitt/miniforge3/envs/flask
kafka                    /Users/stevenhurwitt/miniforge3/envs/kafka
reddit                   /Users/stevenhurwitt/miniforge3/envs/reddit
speedtest                /Users/stevenhurwitt/miniforge3/envs/speedtest
                         /opt/homebrew/Caskroom/miniconda/base
                         /opt/homebrew/anaconda3



In [87]:
! conda init zsh

no change     /Users/stevenhurwitt/miniforge3/condabin/conda
no change     /Users/stevenhurwitt/miniforge3/bin/conda
no change     /Users/stevenhurwitt/miniforge3/bin/conda-env
no change     /Users/stevenhurwitt/miniforge3/bin/activate
no change     /Users/stevenhurwitt/miniforge3/bin/deactivate
no change     /Users/stevenhurwitt/miniforge3/etc/profile.d/conda.sh
no change     /Users/stevenhurwitt/miniforge3/etc/fish/conf.d/conda.fish
no change     /Users/stevenhurwitt/miniforge3/shell/condabin/Conda.psm1
no change     /Users/stevenhurwitt/miniforge3/shell/condabin/conda-hook.ps1
no change     /Users/stevenhurwitt/miniforge3/lib/python3.9/site-packages/xontrib/conda.xsh
no change     /Users/stevenhurwitt/miniforge3/etc/profile.d/conda.csh
no change     /Users/stevenhurwitt/.zshrc
No action taken.


In [88]:
! source ~/.zshrc

In [90]:
# ! conda activate dbx-p40

## directory

In [91]:
base = os.getcwd()

In [92]:
print(base)

/Users/stevenhurwitt/Documents/dealersocket-speedtest


In [93]:
! ls

IDR_Drop     LICENSE      README.md    dealersocket


In [105]:
# ! source ./dealersocker/bin/activate

## python virtualenv

In [103]:
! source ./bin/activate

In [104]:
! python3.7 --version

Python 3.7.10 (77787b8f4c49, May 19 2021, 05:28:43)
[PyPy 7.3.5 with GCC Apple LLVM 12.0.5 (clang-1205.0.22.9)]


In [106]:
! ls

bin           lib           pyvenv.cfg    site-packages


In [107]:
cd ../

/Users/stevenhurwitt/Documents/dealersocket-speedtest


In [108]:
! ls

IDR_Drop     LICENSE      README.md    dealersocket


## pip

In [109]:
! pip --version

pip 21.2.2 from /Users/stevenhurwitt/miniforge3/envs/dealersocket/lib/python3.8/site-packages/pip (python 3.8)


In [110]:
! pip install jupyter

  Using cached qtconsole-5.1.1-py3-none-any.whl (119 kB)


  Using cached QtPy-1.10.0-py2.py3-none-any.whl (54 kB)


In [111]:
! pip install pandas

In [112]:
! pip install requests requests-oauthlib

In [113]:
! pip install pyodbc

  Using cached pyodbc-4.0.32.tar.gz (280 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/stevenhurwitt/miniforge3/envs/dealersocket/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-install-k80mqyxs/pyodbc_f034a809f1de4ae3990c12649d6d4fea/setup.py'"'"'; __file__='"'"'/private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-install-k80mqyxs/pyodbc_f034a809f1de4ae3990c12649d6d4fea/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-wheel-f35qn4p2
       cwd: /private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-install-k80mqyxs/pyodbc_f034a809f1de4ae3990c12649d6d4fea/
  Com

In [114]:
! pip install pyyaml

  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyyaml: filename=PyYAML-5.4.1-cp38-cp38-macosx_11_0_arm64.whl size=45661 sha256=89bc9cd090faa9d9114016db512c95e198a139a183e51e7965f257e2a75b3da3
  Stored in directory: /Users/stevenhurwitt/Library/Caches/pip/wheels/dd/c5/1d/5d7436173d3efd4a14dcb510eb0b29525ecb6b0e41489e716e
Successfully built pyyaml


In [115]:
! pip install beautifulsoup4

In [116]:
! pip install selenium

In [117]:
! pip freeze

appnope @ file:///Users/runner/miniforge3/conda-bld/appnope_1610094717693/work
argon2-cffi @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi_1625821339251/work
async-generator==1.10
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1620387926260/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
beautifulsoup4==4.9.3
bleach @ file:///home/conda/feedstock_root/build_artifacts/bleach_1629908509068/work
blinker==1.4
brotlipy==0.7.0
certifi==2021.5.30
cffi @ file:///Users/runner/miniforge3/conda-bld/cffi_1625835338884/work
chardet @ file:///Users/runner/miniforge3/conda-bld/chardet_1610093454189/work
charset-normalizer @ file:///home/conda/feedstock_root/build_artifacts/charset-normalizer_1626371162869/work
cryptography @ file:///Users/runner/miniforge3/conda-bld/cryptography_1616851494399/work
cx-O

In [119]:
! pip install pytest

  Using cached pytest-6.2.4-py3-none-any.whl (280 kB)
  Using cached pluggy-0.13.1-py2.py3-none-any.whl (18 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached py-1.10.0-py2.py3-none-any.whl (97 kB)
  Using cached iniconfig-1.1.1-py2.py3-none-any.whl (5.0 kB)


In [120]:
! pip freeze > requirements.txt
print("froze requirements.txt")

froze requirements.txt


In [121]:
! ls

IDR_Drop         README.md        requirements.txt
LICENSE          dealersocket


In [ ]:
# ! pip install -r requirements.txt

NameError: name 'requests' is not defined

## etc...

In [74]:
! source ~/.zshrc

## default python

In [124]:
! python --version
! pip --version

Python 3.8.11
pip 21.2.2 from /Users/stevenhurwitt/miniforge3/envs/dealersocket/lib/python3.8/site-packages/pip (python 3.8)


## dbx

In [130]:
! python3.7 --version
! python3.7 -m pip --version

Python 3.7.10 (77787b8f4c49, May 19 2021, 05:28:43)
[PyPy 7.3.5 with GCC Apple LLVM 12.0.5 (clang-1205.0.22.9)]
pip 21.2.4 from /Users/stevenhurwitt/.pyenv/versions/pypy3.7-7.3.5/site-packages/pip (python 3.7)


In [126]:
! java -version

openjdk version "1.8.0_302"
OpenJDK Runtime Environment (Zulu 8.56.0.23-CA-macos-aarch64) (build 1.8.0_302-b08)
OpenJDK 64-Bit Server VM (Zulu 8.56.0.23-CA-macos-aarch64) (build 25.302-b08, mixed mode)


In [127]:
! scala -version

Scala code runner version 2.13.6 -- Copyright 2002-2021, LAMP/EPFL and Lightbend, Inc.


In [133]:
! docker login

Authenticating with existing credentials...
Login Succeeded


In [134]:
! az login

The default web browser has been opened at https://login.microsoftonline.com/common/oauth2/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
    "id": "f55d4ef9-4d7f-4763-8661-9b82de6c08c9",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
      }
    ],
    "name": "zne-udl1-p-ust0-sbc",
    "state": "Enabled",
    "tenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
    "user": {
      "name": "Steven.Hurwitt@bp.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "ea80952e-a476-42d4-aaf4-5457852b0f7e",
    "id": "3dc3f829-f5db-4014-b75f-1718ffb337b2",
    "isDefault": fa

In [129]:
! ls

IDR_Drop         README.md        requirements.txt
LICENSE          dealersocket


## pip error

In [135]:
! source ./dealersocket/bin/activate

In [136]:
! python3.7 -m pip install -r requirements.txt

Processing /Users/runner/miniforge3/conda-bld/appnope_1610094717693/work
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/Users/runner/miniforge3/conda-bld/appnope_1610094717693/work'



In [138]:
! python3.7 --version

Python 3.7.10 (77787b8f4c49, May 19 2021, 05:28:43)
[PyPy 7.3.5 with GCC Apple LLVM 12.0.5 (clang-1205.0.22.9)]


In [139]:
! python3.7 -m pip --version

pip 21.2.4 from /Users/stevenhurwitt/.pyenv/versions/pypy3.7-7.3.5/site-packages/pip (python 3.7)


In [140]:
! python3.7 -m pip install -r requirements.txt

Processing /Users/runner/miniforge3/conda-bld/appnope_1610094717693/work
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/Users/runner/miniforge3/conda-bld/appnope_1610094717693/work'



In [125]:
# ! apt install sql

## pyodbc install error

In [142]:
! python3.7 -m pip install pyodbc

  Using cached pyodbc-4.0.32.tar.gz (280 kB)
  ERROR: Command errored out with exit status 1:
   command: /usr/local/bin/python3.7 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-install-g5tg0hda/pyodbc_98ef1aa9c5b44744a640d915c36035d0/setup.py'"'"'; __file__='"'"'/private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-install-g5tg0hda/pyodbc_98ef1aa9c5b44744a640d915c36035d0/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-wheel-oyiof86v
       cwd: /private/var/folders/bx/9_vlv5nn2_59z73ygpc6z_0m0000gn/T/pip-install-g5tg0hda/pyodbc_98ef1aa9c5b44744a640d915c36035d0/
  Complete output (15 lines):
  running b

In [141]:
import pyodbc

ModuleNotFoundError: No module named 'pyodbc'